In [ ]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
# Load the dataset
dataset = load_dataset("Open-Orca/OpenOrca")

train_data = dataset['train']

print("""Dataset Loaded sucessfully


coverting into DataFrame.....""")

df = pd.DataFrame(train_data)

print("Converted Sucessfully")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset Loaded sucessfully
Now coverting into DataFrame
Converted Sucessfully


In [ ]:
import torch

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
questions = df["question"].tolist()
answers = df["response"].tolist()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Move the input_ids and answer_ids to the selected device
input_ids = tokenizer(questions, return_tensors="pt", padding=False, truncation=True, max_length=64).to(device)
answer_ids = tokenizer(answers, return_tensors="pt", padding=False, truncation=True, max_length=64).to(device)


cuda


In [ ]:
from transformers import GPT2ForSequenceClassification, TrainingArguments, Trainer

model = GPT2ForSequenceClassification.from_pretrained("gpt2").todevice()
training_args = TrainingArguments(
    output_dir="./qa_finetuned_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
).to(device)


In [ ]:

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=input_ids,
    eval_dataset=answer_ids,
).todevice()

trainer.train()


In [ ]:
trainer.save_model("./qa_finetuned_model")